<a href="https://colab.research.google.com/github/rtdatasci/Python/blob/main/smolagent_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install smolagents

In [4]:
from smolagents import CodeAgent, InferenceClientModel

agent=CodeAgent(
    tools=[],
    model=InferenceClientModel()
)

agent.run("Calculate average of the list [32,34,44,2445]")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Calculate average of the list [32,34,44,2445\]                                                                  │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-Next-80B-A3B-Thinking ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  numbers = [32, 34, 44, 2445]                                                                                     
  average = sum(numbers) / len(numbers)                                                                            
  final_answer(average)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: 638.75

[Step 1: Duration 9.73 seconds| Input tokens: 2,044 | Output tokens: 656]

638.75

In [ ]:
# add tools to get external data eg. web
# many tools available in hf doc: smolagents default_tools or
#import specialed tools from hf space hf hub using load_tool by passing repo_id and trust_remove_code=True

from smolagents import CodeAgent, InferenceClientModel, WebSearchTool

agent=CodeAgent(
    tools=[WebSearchTool()],
    model=InferenceClientModel()
)

agent.run("what is the height of the tallest mountain in meters?")


In [ ]:
# add tools to get external data eg. web
# many tools available in hf doc: smolagents default_tools or
#import specialed tools from hf space hf hub using load_tool by passing repo_id and trust_remove_code=True

from smolagents import load_tool

model_downloads_tool <- load_tool(
    repo_id="google/vit-base-patch16-224-in21k",
    trust_remove_code=True
)

agent=CodeAgent(
    tools=[model_downloads_tool, WebSearchTool()],
    model=InferenceClientModel()
)

agent.run("Find the most downloaded image classification model in hugging face")

In [ ]:
# creating custom tools
# eg. a custom tool to read a csv using pandas

from smolagents import tool

# create a tool with @tool decorator
@tool
def check_inventory(product_name:str) -> int:
  """ check available products in the inventory csv"""
  df= pd.read_csv("inventory.csv")
  return df[df["product_name"]==product_name]["quantity"]

from smolagents import CodeAgent

agent = CodeAgent(
    tools=[check_inventory], # custom tool here
    model=InferenceClientModel(),
    additional_authorized_imports=["pandas"]  # allow external packages
)

agent.run("how many large tshirts do you have?")

In [ ]:
# RAG

!pip install langchain_community
!pip install langchain_text_splitters

In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader= PyPDFDirectoryLoader("cooking_docs", mode="Single")
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
)

chunks = splitter.split_documents(documents)

In [ ]:
!pip install langchain_huggingface

In [6]:
!pip install faiss-cpu # for colab env need cpu version to import faiss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 68.8 MB/s eta 0:00:00


In [ ]:
!mkdir -p cooking_docs
print("Directory 'cooking_docs' created.")
# Now, manually upload your PDF files into the 'cooking_docs' folder using the Colab file browser (folder icon on the left).
# After uploading, you can re-run cell 'DnTPlAhuXaBG'.

In [ ]:
# create a vector store FAISS (similarity search)
# HFendpoint embedding makes sure the query term is matched even if the query term for example error15 in dishwasher might appear in many parts of the manual

from langchain_huggingface import HuggingFaceEndpointEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEndpointEmbeddings(
    model="BAAI/bge-base-en-v1.5",
    task="feature-extraction"
)

vector_store = FAISS.from_documents(chunks, embeddings)
## will error out because chunks is empty which is because there is no 'cooking_docs"


In [ ]:
#query the vector store
query = "how do i cook salmon in air fryer?"

# similarity search (get all relevant sections)
relevant_docs = vector_store.similarity_search(query, k=3)

# create a context string
context = "\n\n".join([doc.page_content for doc in relevant_docs])


In [ ]:
# agentic rag
# retrievel plus reasoning

# decorator such as @tool lack memory between queries
# use class Tool instead

from smolagents import Tool

class RecipeSearchTool(Tool):
  name="recipe_search",
  description="searching cooking documentation for recipes and ingredients",
  inputs={
      "query":{
          "type":"string",
          "description":"natural language cooking query",
      }
  },
  output_type="string",
  def __init__(self, vector_store, k=6):  # how many documets to retrieve eg. 6
    super().__init__()
    self.vector_store = vector_store
    self.k = k

    # incase no response
  def forward(self, query):
    docs = self.vector_store.similarity_search(query, k=self.k)
    return "\n\n".join(doc.page_content for doc in docs) or "Nothing found"

# create agent with this tool
recipe_search = RecipeSearchTool(vector_store)

agent = CodeAgent(
    tools=[recipe_search],
    model=model,
    instructions="you are a helpful cooking assistant. Be thorough. If initial result is incomplete, try different search terms",
    verbosity_level=1,
    max_steps=8

)
result = agent.run("If the AC isn’t cooling and shows error E1, what should I check and what’s the next step?")
print(result)


In [ ]:
#multi-step agent
# incoporate planning agent to force a pause and re-think, evaluate what it learned
# call back function to make agent do certain custom things after certain steps
# callback can be planning (checking) or action (doing) something
# callbacks for human checkpoints, checking login status, checking how much token used, etc


from smolagents import PlanningStep, ActionStep

agent = CodeAgent(
    tools=[document_search_tool],
    model = model,
    step_callbacks= {
        PlanningStep: planning_callback,
        ActionStep: action_callback
    }
)

In [ ]:
# multi agent with multiple specialist
# example manager_aget with deepseek reasoning to coordinate, resume_agent with webtools, company_agent with custom tools, etc
# manager_agent running maanaged_agents: resume, company, etc


# school application agents
# School research specialist
school_agent = CodeAgent(
    # Assign a list of tools the agent can use
    tools=[WebSearchTool()],
    model=model,
    # Set the agent's unique name identifier
    name="school_research_agent",
    description="Expert in researching universities, programs, and admission requirements"
)

# Essay writing specialist
essay_agent = CodeAgent(
    tools=[WebSearchTool()],
    # Provide the model used to generate responses
    model=model,
    name="essay_writing_agent",
    # Write a short description of the agent's area of expertise
    description="Expert in crafting compelling college application essays and personal statements"
)





In [ ]:
# managing memory
# each agent.run() starts fresh and has no memory
# use reset=False to continue follow up conversation with memory of old results
# eg. conversing with booking travel agent
# Step 1: Tell the agent your flight date
travel_agent.run("My Tokyo flight confirmation code is ZX9Q2L.")

# Step 2: Confirm the agent remember when passing the correct reset parameter
follow_up = "What’s my Tokyo flight confirmation code?"
response = travel_agent.run(follow_up, reset=False)

print(response)

# debug with essay_agent.memory.return_full_code()
executed_code = travel_agent.memory.return_full_code()

# check what agent is doing essay_agent.memory.get_succinct_steps()
# can also save steps to flat json for analysis and regression testing



In [ ]:
# agent answer validation
# creating validation rule functions to check eg. response is at least 200 char long
# also using another llm to validate with validation prompt """ reasoning_process{}, agents final answer{}. does the answer round reasonable. respond true or false."""

def check_answer_length(final_answer, agent_memory):
    # Check if answer contains less than 200 characters
    if len(str(final_answer)) < 200:
        raise Exception("The answer is too short. Please include more details.")
    # Return True if check passes
    return True


real_estate_agent = CodeAgent(
    tools=[],
    model=model,
    # Create the agent with answer length validation
    final_answer_checks=[check_answer_length],
    verbosity_level=2
)

# Run the agent with a short prompt
response = real_estate_agent.run("Suggest a neigborhood for a couple moving to Austin.")
print(response)



